# Mini Project: Neural Networks for Healthcare Readmission Prediction

## Problem Overview

Hospital readmissions within 30 days are a major concern for healthcare providers, impacting both patient outcomes and hospital costs.
The objective of this project is to predict whether a patient will be readmitted within 30 days using an Artificial Neural Network (ANN).

Target variable:

readmitted_30_days

0 → Not readmitted

1 → Readmitted

## 1. Data Understanding & Preprocessing

### 📊 Loading Dataset

In [31]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/4721678-healthcare_readmission_dataset.csv')
df.head()

,patient_id,age,bmi,systolic_bp,diastolic_bp,heart_rate,glucose_level,num_previous_admissions,length_of_stay,chronic_condition,smoker,insurance_type,medication_count,followup_appointment,readmitted_30_days
0,PAT00001,69,31.9,127.0,76.0,85.0,115.0,3,3,cardiac,yes,public,5,no,1
1,PAT00002,32,16.5,102.0,70.0,84.0,133.0,1,8,diabetes,no,private,4,no,1
2,PAT00003,89,32.8,158.0,83.0,77.0,95.0,2,12,respiratory,no,self-pay,7,yes,1
3,PAT00004,78,33.9,105.0,82.0,86.0,151.0,0,17,diabetes,no,private,1,no,1
4,PAT00005,38,25.5,120.0,101.0,81.0,108.0,1,14,none,yes,private,10,yes,0


### 🔍 Dataset Shape & Info

In [32]:
print(df.shape)
print('-'*50)
print(df.info())


(5000, 15)
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   patient_id               5000 non-null   object 
 1   age                      5000 non-null   int64  
 2   bmi                      5000 non-null   float64
 3   systolic_bp              5000 non-null   float64
 4   diastolic_bp             5000 non-null   float64
 5   heart_rate               5000 non-null   float64
 6   glucose_level            5000 non-null   float64
 7   num_previous_admissions  5000 non-null   int64  
 8   length_of_stay           5000 non-null   int64  
 9   chronic_condition        5000 non-null   object 
 10  smoker                   5000 non-null   object 
 11  insurance_type           5000 non-null   object 
 12  medication_count         5000 non-null   int64  
 13  followup_appoint

Observation:

*   Dataset contains both numerical and categorical features
*   Target variable is binary
*   Suitable for binary classification using ANN

### ❓ Missing Values Check

In [33]:
df.isnull().sum()

,0
patient_id,0
age,0
bmi,0
systolic_bp,0
diastolic_bp,0
heart_rate,0
glucose_level,0
num_previous_admissions,0
length_of_stay,0
chronic_condition,0


Handling Strategy:

*   Numerical columns → Median imputation
*   Categorical columns → Mode imputation

In [34]:
from sklearn.impute import SimpleImputer

num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])


### 🔁 Encoding Categorical Variables

In [35]:
df = pd.get_dummies(df, drop_first=True)

### 📐 Feature Scaling

In [36]:
from sklearn.preprocessing import StandardScaler

X = df.drop("readmitted_30_days", axis=1)
y = df["readmitted_30_days"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 🔀 Train-Test Split

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size = 0.2, random_state = 42, stratify = y
)

## 2. Forward Propagation & Loss Intuition

Forward Propogation:

For a neuron:
    
    z = w1.x1 + w2.x2 + b
    a = σ(z)
  where:
    σ(z) = 1 / (1 + e^(-z))


In [38]:
import math

x1, x2 = 0.6, 0.8
w1, w2 = 0.4, 0.7
b = 0.2

z = (w1*x1) + (w2*x2) + b
a = 1 / (1 + math.exp(-z))

z,a

(1.0, 0.7310585786300049)

Loss Function Comparison:

1. Mean Squared Error:

   *   Penalizes large errors heavily
   *   Not ideal for classification

2. Binary Cross-Entropy (BCE):

    Loss = y.log(p) + (1-y).log(1-p)

->Binary Cross-Entropy is better suited for probabilistic    outputs in classification problems, hence used in this project.




## 3. Baseline ANN Model

### 🧠 Model Architecture
*   Input layer
*   2 Hidden layers (ReLU)
*   Output layer (Sigmoid)

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
# Compile
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [41]:
# Training
history = model.fit(
    X_train, y_train,
    validation_split = 0.2,
    epochs = 10,
    batch_size = 32,
    verbose = 1
)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5197 - loss: 0.8374 - val_accuracy: 0.5638 - val_loss: 0.6948
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9279 - loss: 0.3136 - val_accuracy: 0.5775 - val_loss: 0.7062
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9958 - loss: 0.0625 - val_accuracy: 0.5925 - val_loss: 0.7416
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9999 - loss: 0.0109 - val_accuracy: 0.5900 - val_loss: 0.7655
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.5888 - val_loss: 0.7819
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.5900 - val_loss: 0.7948
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.5950 - val_loss: 0.8058
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 9.7778e-04 - val_accuracy

In [42]:
# Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5956 - loss: 0.8371


0.5889999866485596

## 4. Hyperparameter Tuning & Regularization

### Dropout & L2 Regularization

In [43]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [44]:
model_reg = Sequential([
    Dense(64, activation='relu', kernel_regularizer=l2(0.001), input_shape = (X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Re-Compile & Train

In [45]:
model_reg.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [46]:
history_reg = model_reg.fit(
    X_train, y_train,
    validation_split = 0.2,
    epochs = 50,
    batch_size = 64,
    verbose=1
)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.4959 - loss: 1.0241 - val_accuracy: 0.5625 - val_loss: 0.8136
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7207 - loss: 0.6700 - val_accuracy: 0.6075 - val_loss: 0.7663
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8863 - loss: 0.4316 - val_accuracy: 0.6150 - val_loss: 0.7725
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9555 - loss: 0.2620 - val_accuracy: 0.5738 - val_loss: 0.7749
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9836 - loss: 0.1701 - val_accuracy: 0.5700 - val_loss: 0.7683
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9936 - loss: 0.1297 - val_accuracy: 0.5650 - val_loss: 0.7651
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9921 - loss: 0.1214 - val_accuracy: 0.5700 - val_loss: 0.7598
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9962 - loss: 0.1033 - val_accuracy: 0.5675 - v

In [47]:
reg_test_loss, reg_test_acc = model_reg.evaluate(X_test, y_test)
reg_test_acc

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6511 - loss: 0.6433


0.6359999775886536

## 5. Advanced Training Strategies & Report

In [48]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

In [49]:
model_adv = Sequential([
    Dense(64, kernel_regularizer=l2(0.001),
          input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.3),

    Dense(32, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [50]:
# Compile
model_adv.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [51]:
# Early Stopping:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    restore_best_weights = True
)

In [52]:
# Learning Rate Scheduling
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.5,
    patience = 3
)

### Final Training

In [53]:
history_adv = model_adv.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stop, lr_schedule],
    verbose=1
)


Epoch 1/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.5084 - loss: 0.9686 - val_accuracy: 0.6025 - val_loss: 0.7325 - learning_rate: 0.0010
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7319 - loss: 0.6247 - val_accuracy: 0.6350 - val_loss: 0.7145 - learning_rate: 0.0010
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8974 - loss: 0.4110 - val_accuracy: 0.6463 - val_loss: 0.6950 - learning_rate: 0.0010
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9667 - loss: 0.2650 - val_accuracy: 0.6300 - val_loss: 0.6871 - learning_rate: 0.0010
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9878 - loss: 0.1797 - val_accuracy: 0.6288 - val_loss: 0.6939 - learning_rate: 0.0010
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9950 - loss: 0.1381 - val_accuracy: 0.6025 - val_loss: 0.7172 - learning_rate: 0.0010
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9965 - loss: 0.1132 - 

In [54]:
test_loss, test_acc = model_adv.evaluate(X_test, y_test)
test_acc


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6289 - loss: 0.6912


0.6000000238418579

## Final Results & Insights
### Best Model Configuration

*   2 Hidden Layers
*   ReLU Activations
*   Dropout (0.3)
*   L2 Regularization (0.001)
*   Early Stopping, LR Scheduler & batch normalization

### Practical Recommendations

*   Patients with high prior admissions and longer length of stay show higher readmission risk
*   Model can assist hospitals in targeted follow-up care
*   Supports cost reduction & patient outcome improvement


## Conclusion
This project demonstrates how Artificial Neural Networks can be effectively applied to healthcare data for predicting patient readmission.
Using proper preprocessing, regularization, and advanced training strategies significantly improves model stability and generalization.